<a href="https://colab.research.google.com/github/jayrom/enterprise_challenge_reply/blob/main/sprint_4/src/train_complex_data_generation_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulação de dados de sensores industriais
*Dataset de manutenção (rotulado)*

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime

# ==============================================================================
# 1. FUNÇÃO PRINCIPAL PARA GERAR O CICLO DE VIDA DE UM MOTOR
# ==============================================================================
def gerar_ciclo_de_vida_motor(device_id: str, n_leituras: int, start_time: datetime,
                              modo_falha: str = 'nenhuma', inicio_falha_percent: float = 0.7):
    """
    Gera uma série temporal completa para um único motor, desde o início da operação
    até o final do período de leituras, simulando um modo de falha específico.

    Args:
        device_id (str): Identificador único do motor (ex: 'motor_1').
        n_leituras (int): Número total de registros a serem gerados para este motor.
        start_time (datetime): Timestamp inicial para a primeira leitura.
        modo_falha (str): 'nenhuma', 'desgaste_rolamento' ou 'desbalanceamento'.
        inicio_falha_percent (float): Ponto no ciclo de vida (0.0 a 1.0) onde a falha começa.

    Returns:
        pd.DataFrame: DataFrame com a série temporal completa para o motor.
    """
    # --- Configurações Iniciais ---
    intervalo_minutos = 10
    timestamps = pd.to_datetime([start_time + pd.Timedelta(minutes=i*intervalo_minutos) for i in range(n_leituras)])

    # --- Geração dos Dados Base (Estado Normal) ---
    temperature = np.random.normal(31, 2, n_leituras)
    current = np.random.normal(1.6, 0.1, n_leituras)
    vib_x = np.random.normal(0, 0.05, n_leituras)
    vib_y = np.random.normal(0, 0.05, n_leituras)
    vib_z = np.random.normal(9.8, 0.05, n_leituras) # Inclui gravidade

    # --- Simulação da Falha Progressiva (se houver) ---
    days_to_failure = np.zeros(n_leituras)
    failure_mode_list = ['normal'] * n_leituras

    if modo_falha != 'nenhuma':
        start_failure_idx = int(n_leituras * inicio_falha_percent)

        for i in range(start_failure_idx, n_leituras):
            progress = (i - start_failure_idx) / (n_leituras - start_failure_idx)

            failure_mode_list[i] = modo_falha
            days_remaining = (n_leituras - 1 - i) * intervalo_minutos / (60 * 24)
            days_to_failure[i] = max(0, days_remaining)

            if modo_falha == 'desgaste_rolamento':
                temperature[i] += 15 * np.sin(progress * np.pi) + np.random.normal(0, 1)
                current[i] += 5 * np.sin(progress * np.pi) + np.random.normal(0, 0.5)
                vib_x[i] += 1.0 * np.sin(progress * np.pi) + np.random.normal(0, 0.1)
                vib_y[i] += 1.2 * np.sin(progress * np.pi) + np.random.normal(0, 0.1)
                vib_z[i] += 0.3 * np.sin(progress * np.pi) + np.random.normal(0, 0.05)

            elif modo_falha == 'desbalanceamento':
                if progress > 0.3:
                    temperature[i] += 10 * np.sqrt(progress - 0.3) + np.random.normal(0, 1.2)
                    current[i] += 4 * np.sqrt(progress - 0.3) + np.random.normal(0, 0.6)
                vib_x[i] += 0.5 * np.exp(progress * 1.5) + np.random.normal(0, 0.10)
                vib_y[i] += 2.5 * np.exp(progress * 1.5) + np.random.normal(0, 0.20)
                vib_z[i] += 0.2 * np.exp(progress * 1.5) + np.random.normal(0, 0.08)

    # --- Finalização ---
    vibration_magnitude = np.sqrt(vib_x**2 + vib_y**2 + vib_z**2)

    df_motor = pd.DataFrame({
        'timestamp': timestamps,
        'device_id': device_id,
        'temperature_c': temperature.round(2),
        'current_amps': current.round(2),
        'vibration_magnitude_mss': vibration_magnitude.round(2),
        'days_to_failure': days_to_failure.round(2),
        'failure_mode': failure_mode_list
    })

    return df_motor

# ==============================================================================
# 2. CONFIGURAÇÃO E EXECUÇÃO DA SIMULAÇÃO
# ==============================================================================
if __name__ == '__main__':
    # Seed para reprodutibilidade
    np.random.seed(42)
    random.seed(42)

    # --- Painel de Controle da Simulação ---
    # Aqui definimos quantos motores de cada tipo queremos criar
    CONFIGURACAO_SIMULACAO = [
        # --- Sua sugestão: Uma falha de rolamento que começa bem no início ---
        {'tipo': 'desgaste_rolamento', 'n_leituras': 400, 'inicio_falha': 0.1, 'device_id': 'motor_A'},

        # --- Uma falha de desbalanceamento que começa no meio do ciclo de vida ---
        {'tipo': 'desbalanceamento', 'n_leituras': 500, 'inicio_falha': 0.5, 'device_id': 'motor_B'},

        # --- Uma falha de rolamento que começa mais para o final ---
        {'tipo': 'desgaste_rolamento', 'n_leituras': 600, 'inicio_falha': 0.8, 'device_id': 'motor_C'},

        # --- Um motor que desenvolve desbalanceamento, mas a simulação acaba antes da quebra ---
        {'tipo': 'desbalanceamento', 'n_leituras': 300, 'inicio_falha': 0.6, 'device_id': 'motor_D'},

        # --- Dois motores que operam normalmente durante todo o período ---
        {'tipo': 'nenhuma', 'n_leituras': 500, 'inicio_falha': None, 'device_id': 'motor_E'},
        {'tipo': 'nenhuma', 'n_leituras': 450, 'inicio_falha': None, 'device_id': 'motor_F'},
    ]

    print("Iniciando a geração do dataset de treinamento robusto...")

    lista_de_dataframes = []
    timestamp_atual = pd.Timestamp('2024-01-01 00:00:00')

    # Loop principal que "fabrica" os motores
    for config in CONFIGURACAO_SIMULACAO:
        print(f"  -> Gerando ciclo de vida para o device '{config['device_id']}'...")

        df_motor = gerar_ciclo_de_vida_motor(
            device_id=config['device_id'],
            n_leituras=config['n_leituras'],
            start_time=timestamp_atual,
            modo_falha=config['tipo'],
            inicio_falha_percent=config['inicio_falha']
        )

        lista_de_dataframes.append(df_motor)

        # Define o tempo de início para o próximo motor, garantindo a continuidade cronológica
        timestamp_atual = df_motor.iloc[-1]['timestamp'] + pd.Timedelta(minutes=10)

    # Concatena todos os dataframes em um único dataset histórico
    df_final = pd.concat(lista_de_dataframes, ignore_index=True)

    # Salva o arquivo
    nome_arquivo = 'training_dataset_robusto.csv'
    df_final.to_csv(nome_arquivo, index=False)

    print("\n----------------------------------------------------")
    print("Dataset de treinamento robusto gerado com sucesso!")
    print(f"Arquivo salvo como: '{nome_arquivo}'")
    print(f"Total de registros gerados: {len(df_final)}")
    print("\nDistribuição dos modos de falha no dataset final:")
    print(df_final['failure_mode'].value_counts(normalize=True).round(2))
    print("\nResumo do dataset:")
    print(df_final.head(3))
    print("...")
    print(df_final.tail(3))

Iniciando a geração do dataset de treinamento robusto...
  -> Gerando ciclo de vida para o device 'motor_A'...
  -> Gerando ciclo de vida para o device 'motor_B'...
  -> Gerando ciclo de vida para o device 'motor_C'...
  -> Gerando ciclo de vida para o device 'motor_D'...
  -> Gerando ciclo de vida para o device 'motor_E'...
  -> Gerando ciclo de vida para o device 'motor_F'...

----------------------------------------------------
Dataset de treinamento robusto gerado com sucesso!
Arquivo salvo como: 'training_dataset_robusto.csv'
Total de registros gerados: 2750

Distribuição dos modos de falha no dataset final:
failure_mode
normal                0.69
desgaste_rolamento    0.17
desbalanceamento      0.13
Name: proportion, dtype: float64

Resumo do dataset:
            timestamp device_id  temperature_c  current_amps  \
0 2024-01-01 00:00:00   motor_A          31.99          1.44   
1 2024-01-01 00:10:00   motor_A          30.72          1.54   
2 2024-01-01 00:20:00   motor_A         

### Importar módulos

### Configurações iniciais da simulação

### Parâmetros de dados normais (média e desvio padrão para ruído)

### Parâmetros de falha (taxa de aumento por dia)

### Geração dos dados